In [1]:
import qrcode
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image 
import PIL
from skimage.measure import compare_ssim
import cv2
from numpy import *
import pywt
from resizeimage import resizeimage
import skimage.color

In [2]:
from scipy.optimize import rosen, differential_evolution
bounds = [(0,2), (0, 2), (0, 2), (0, 2), (0, 2)]
result = differential_evolution(rosen, bounds)

In [3]:
result

     fun: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 44631
     nit: 594
 success: True
       x: array([1., 1., 1., 1., 1.])

In [ ]:
result?

In [4]:
# -*- coding: utf-8 -*-
# @Time    : 14/04/2017 14:46
# @Author  : Luke
# @Software: PyCharm

import cv2
import pywt
import numpy as np


class Components():
    Coefficients = []
    U = None
    S = None
    V = None


class watermarking():
    """
    :param watermark_path:
    :param ratio:
    :param wavelet:
    :param level:
    """
    def __init__(self, watermark_path="watermark1.jpg", ratio=0.1, wavelet="haar",
                 level=2):
        self.level = level
        self.wavelet = wavelet
        self.ratio = ratio
        self.shape_watermark = cv2.imread(watermark_path,0).shape  
        self.W_components = Components()
        self.img_components = Components()
        self.W_components.Coefficients, self.W_components.U, \
        self.W_components.S, self.W_components.V = self.calculate(watermark_path)
        #self.W_ndarr = cv2.imread(watermark_path,0)
        #print("watermark", self.W_ndarr.shape)

    def calculate(self, img):
        '''
        To calculate the Coefficients and SVD components.
        :param img: should be a numpy array or the path of the image.
        '''
        if isinstance(img, str):
            img = cv2.imread(img, 0)
        Coefficients = pywt.wavedec2(img, wavelet=self.wavelet, level=self.level)
        self.shape_LL = Coefficients[0].shape
        U, S, V = np.linalg.svd(Coefficients[0])
        return Coefficients, U, S, V

    def diag(self, s):
        '''
        To recover the singular values to be a matrix.
        :param s: a 1D numpy array
        '''
        S = np.zeros(self.shape_LL)
        row = min(S.shape)
        S[:row, :row] = np.diag(s)
        return S

    def recover(self, name):
        '''
        To recover the image from the svd components and DWT
        :param name:
        '''
        components = eval("self.{}_components".format(name))
        s = eval("self.S_{}".format(name))
        components.Coefficients[0] = components.U.dot(self.diag(s)).dot(components.V)
        return pywt.waverec2(components.Coefficients, wavelet=self.wavelet)

    def watermark(self, img="lena.jpg", path_save=None):
        '''
        This is the main function for image watermarking.
        :param img: image path or numpy array of the image.
        '''
        if not path_save:
            path_save = "watermarked_" + img
        self.path_save = path_save
        self.img_components.Coefficients, self.img_components.U, \
        self.img_components.S, self.img_components.V = self.calculate(img)
        self.embed()
        img_rec = self.recover("img")   #watermarked image
        cv2.imwrite(path_save, img_rec)

    def extracted(self, image_path=None, ratio=None, extracted_watermark_path = None):
        '''
        Extracted the watermark from the given image.
        '''
        if not extracted_watermark_path:
            extracted_watermark_path = "watermark_extracted.png"
        if not image_path:
            image_path = self.path_save
        img = cv2.imread(image_path,0)
        img = cv2.resize(img, self.shape_watermark)
        img_components = Components() #watermarked image
        img_components.Coefficients, img_components.U, img_components.S, img_components.V = self.calculate(img)
        ratio_ = self.ratio if not ratio else ratio
        self.S_W = (img_components.S - self.img_components.S) / ratio_
        watermark_extracted = self.recover("W")
        cv2.imwrite(extracted_watermark_path, watermark_extracted)

    def embed(self):
        self.S_img = self.img_components.S + self.ratio * self.W_components.S * \
                                             (self.img_components.S.max() / self.W_components.S.max())
        
    def psnr_cal(self, img1="lena.png" , img2="watermark_extracted.png"):
        im1 = cv2.imread('lena.png')
        im2 = cv2.imread('watermark_extracted.png')
        # Compute PSNR over tf.uint8 Tensors.
        psnr1 = skimage.metrics.peak_signal_noise_ratio(im1, im2)
        print(psnr1)

        

In [5]:
water_marking = watermarking()
water_marking.watermark()
water_marking.extracted()
water_marking.psnr_cal()

12.502122176868875


In [13]:
a = (1,2)
b = (3,4)
c = (5,6)
k = []
k.append(a)
k.append(b)
k.append(c)

In [14]:
k[0]

(1, 2)

In [15]:
k[1][1]

4

In [23]:
np.linspace(0, 1, num = 30, endpoint = False)

array([0.        , 0.03333333, 0.06666667, 0.1       , 0.13333333,
       0.16666667, 0.2       , 0.23333333, 0.26666667, 0.3       ,
       0.33333333, 0.36666667, 0.4       , 0.43333333, 0.46666667,
       0.5       , 0.53333333, 0.56666667, 0.6       , 0.63333333,
       0.66666667, 0.7       , 0.73333333, 0.76666667, 0.8       ,
       0.83333333, 0.86666667, 0.9       , 0.93333333, 0.96666667])

In [22]:
import numpy as np